# 在 Bedrock AgentCore Runtime 中部署 Strands Agent

本研讨会演示如何使用 Amazon Bedrock AgentCore Runtime 部署 Strands Agents，集成多种工具，包括 MCP 服务器、代码解释器和自定义函数，以实现全面的 AI 代理功能。

## 概述

在本实验中，您将：
- 部署具有多个集成工具的 Strands Agent
- 使用之前实验中的 MCP 服务器作为代理工具
- 集成 Bedrock AgentCore Code Interpreter
- 使用各种场景测试已部署的代理
- 了解可观测性和监控

## 先决条件

开始本实验之前，请确保您具备：
- 已完成实验 2（EXA API 密钥存储在 AgentCore Identity 中）和实验 3（MCP 服务器部署）
- 已配置 AWS 凭证（IAM 角色或环境变量）
- 已安装所需的 Python 包
- 对 Strands Agents 和 Bedrock AgentCore 概念有基本了解

如果您没有在假设 IAM 角色的环境中运行，请将您的 AWS 凭证设置为环境变量：

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

为 Strands Agents、MCP 集成和 Bedrock AgentCore SDK 安装所需的包：

In [ ]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore bedrock-agentcore-starter-toolkit

## 从实验 3 检索配置以连接已部署的 MCP 服务器

从上一个实验中加载 MCP 服务器部署详细信息和身份验证配置。

### 检索的组件：
- **Cognito 客户端 ID**：用于与已部署的 MCP 服务器进行身份验证
- **Cognito 池 ID**：用户池配置
- **MCP Runtime ARN**：已部署的 MCP 服务器端点
- **MCP Runtime ID**：运行时标识符
- **ECR 存储库名称**：容器注册表信息

In [ ]:
%store -r cognito_client_id
%store -r cognito_pool_id
%store -r mcp_runtime_arn
%store -r mcp_runtime_id
%store -r mcp_ecr_repo_name

## 什么是带有 Bedrock AgentCore Runtime 的 Strands Agent？

Strands Agents 为构建具有多种工具集成的 AI 代理提供了强大的框架。当与 Bedrock AgentCore Runtime 一起部署时，您可以获得：

- **可扩展部署**：具有自动扩展功能的托管基础设施
- **安全身份验证**：与 Cognito 集成的内置安全性
- **可观测性**：CloudWatch 集成用于监控和调试
- **多工具集成**：在 AgentCore Runtime 中结合 MCP 服务器、代码解释器和浏览器

本实验演示创建一个能够执行网络搜索、执行代码和处理自定义业务逻辑的综合代理。

## 创建多工具 Strands Agent

让我们创建一个集成之前实验中多种工具的 Strands Agent：

### 工具集成概述：
- **自定义天气工具**：简单演示功能
- **计算器工具**：数学运算
- **代码解释器**：安全执行 Python 代码（实验 1）
- **EXA MCP 工具**：来自远程 MCP 服务器的网络搜索（实验 2）
- **已部署的 MCP 工具**：来自 AgentCore Runtime MCP 服务器的网络搜索（实验 3）

In [ ]:
# Lab 2: Check Bedrock AgentCore Identity for securely storing EXA API Key in Lab 3, or create if not exist
from bedrock_agentcore.services.identity import IdentityClient
import boto3

# !-------- UPDATE THE EXA API KEY HERE  --------!
EXA_API_KEY = <YOUR EXA API KEY> 

region = boto3.session.Session().region_name

#Configure API Key Provider
identity_client = IdentityClient(region=region)

try:
    api_key_provider = identity_client.create_api_key_credential_provider({
        "name": "exa-apikey-provider",
        "apiKey": EXA_API_KEY # Replace it with the API key you obtain from the external application vendor, e.g., OpenAI
    })
    print("Created AgentCore Identity API Key Credential Provider.")
except Exception as e:
    print("AgentCore Identity API Key Credential Provider already exist.")

In [ ]:
# Lab 3: Copy the configuration values to Strands Agents setup below for deployed MCP Server connection
print(f"MCP Server AgentCore Runtime ARN: {mcp_runtime_arn}")
print(f"Cognito Client ID: {cognito_client_id}")

⚠️ **Note**: Update the MCP_RUNTIME_ARN and COGNITO_CLIENT_ID variables with values from your Lab 3 deployment.

In [ ]:
from strands import Agent, tool
from strands_tools import calculator
from strands_tools.code_interpreter import AgentCoreCodeInterpreter
from bedrock_agentcore.identity.auth import requires_api_key
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import boto3

# !-------- UPDATE THE CONFIGURATION VALUES FROM LAB 3  --------!
MCP_RUNTIME_ARN = <COPY MCP_RUNTIME_ARN IN LAB 3 HERE>
COGNITO_CLIENT_ID = <COPY COGNITO_CLIENT_ID IN LAB 3 HERE>

# Create a custom weather tool for demonstration
@tool
def weather(city: str) -> str:
    """Get weather information for a city
    Args:
        city: City or location name
    """
    return f"Weather for {city}: Sunny, 35°C"

# Lab 1: Bedrock AgentCore Code Interpreter
# -----------------------------------------
print("Setting up Code Interpreter...")
agentcore_code_interpreter = AgentCoreCodeInterpreter()


# Lab 2: Bedrock AgentCore Identity - Secure API key for Remote MCP Server
# ------------------------------------------------------------------------
@requires_api_key(provider_name="exa-apikey-provider")
def need_api_key(*, api_key: str):
    #global EXA_API_KEY
    print(f'received api key for async func: {api_key}')
    return api_key
EXA_API_KEY = need_api_key()

# Connect to the EXA MCP server
exa_server = MCPClient(lambda: streamablehttp_client(f"https://mcp.exa.ai/mcp?exaApiKey={EXA_API_KEY}"))


# Lab 3: Bedrock AgentCore Runtime - MCP Server Hosting
# -----------------------------------------------------
print("Setting up MCP server authentication...")

region = boto3.session.Session().region_name
cognito_client = boto3.client('cognito-idp', region_name=region)

# Get bearer token (access token) from Cognito Auth 
auth_response = cognito_client.initiate_auth(
    ClientId=COGNITO_CLIENT_ID,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': 'testuser',
        'PASSWORD': 'MyPassword123!'
    }
)
bearer_token = auth_response['AuthenticationResult']['AccessToken']

# Configure deployed MCP server connection
encoded_arn = MCP_RUNTIME_ARN.replace(':', '%3A').replace('/', '%2F')
mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
headers = {"Authorization": f"Bearer {bearer_token}"}

# Connect to the deployed MCP server
print("Connecting to MCP server...")
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url, headers))


# Create and test the comprehensive Strands Agent
with exa_server, websearch_server:
    mcp_tools = (exa_server.list_tools_sync() + websearch_server.list_tools_sync())
    print(f"Available MCP tools: {[tool.tool_name for tool in mcp_tools]}")

    agent = Agent(
        model="us.amazon.nova-pro-v1:0",
        system_prompt = """You are a helpful assistant that provides concise responses.
                        If web search, always use websearch tool first.
                        """,
        #system_prompt="""你是一位乐于助人的助手，能提供简洁的回复。
        #                如果要进行网页搜索，请务必先使用websearch工具。
        #                """,
        tools=[weather, calculator, agentcore_code_interpreter.code_interpreter] + mcp_tools,
    )

    print("\n=== Testing Agent Capabilities ===")
    print("\n1. Testing custom weather tool:")
    #agent("How is the weather in HK?")
    agent("北京的天气怎么样？")
    
    print("\n2. Testing calculator tool:")
    agent("2+2=")
    
    print("\n3. Testing web search via MCP:")
    #agent("Search for a coffee maker on amazon.com and extract details of the first one")
    agent("在 amazon.com 上搜索咖啡机并提取第一款咖啡机的详细信息")
    
    print("\n4. Testing code interpreter:")
    #agent("Tell me the largest random prime number between 1 and 100, which is less than 84 and more that 9")
    agent("告诉我 1 到 100 之间最大的随机素数，小于 84 且大于 9")

## 将 Strands Agent 部署到 Bedrock AgentCore Runtime

创建我们的 Strands Agent 的可部署版本并将其部署为托管服务。

### 部署过程：
1. 创建带有代理配置的 Python 文件
2. 设置带有依赖项的 requirements.txt
3. 配置带有身份验证的 AgentCore Runtime
4. 使用 CodeBuild 进行容器化部署

### 步骤 1：创建带有代理配置的 Python 文件

创建定义我们的 Strands Agent 及其所有集成工具的主 Python 文件。该文件将作为容器化服务部署到 Bedrock AgentCore Runtime。
![bedrock-agentcore-runtime-launch](images/runtime-launch.png)

**部署要求：**
要将代理部署到 AgentCore Runtime，我们需要：
- 导入 Runtime App：`from bedrock_agentcore.runtime import BedrockAgentCoreApp`
- 初始化 App：`app = BedrockAgentCoreApp()`
- 使用 `@app.entrypoint` 装饰调用函数
- 让 AgentCore Runtime 通过 `app.run()` 控制执行

In [ ]:
# Copy the values in Lab 3 to Strands Agents setup below
print(f"MCP Server AgentCore Runtime ARN: {mcp_runtime_arn}")
print(f"Cognito Client ID: {cognito_client_id}")

⚠️ **Note**: Update the MCP_RUNTIME_ARN and COGNITO_CLIENT_ID variables with values from your Lab 3 deployment.

In [ ]:
%%writefile strands_agent.py
from strands import Agent, tool
from strands_tools import calculator
from strands_tools.code_interpreter import AgentCoreCodeInterpreter
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import boto3
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# !-------- UPDATE THE CONFIGURATION VALUES FROM LAB 3  --------!
MCP_RUNTIME_ARN = <COPY MCP_RUNTIME_ARN IN LAB 3 HERE>
COGNITO_CLIENT_ID = <COPY COGNITO_CLIENT_ID IN LAB 3 HERE>

# Create a custom weather tool for demonstration
@tool
def weather(city: str) -> str:
    """Get weather information for a city
    Args:
        city: City or location name
    """
    return f"Weather for {city}: Sunny, 35°C"

# Lab 1: Bedrock AgentCore Code Interpreter
# -----------------------------------------
print("Setting up Code Interpreter...")
agentcore_code_interpreter = AgentCoreCodeInterpreter()


# Lab 3: Bedrock AgentCore Runtime - MCP Server Hosting
# -----------------------------------------------------
print("Setting up MCP server authentication...")

region = boto3.session.Session().region_name
cognito_client = boto3.client('cognito-idp', region_name=region)

# Get bearer token (access token) from Cognito Auth 
auth_response = cognito_client.initiate_auth(
    ClientId=COGNITO_CLIENT_ID,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': 'testuser',
        'PASSWORD': 'MyPassword123!'
    }
)
bearer_token = auth_response['AuthenticationResult']['AccessToken']

# Configure deployed MCP server connection
encoded_arn = MCP_RUNTIME_ARN.replace(':', '%3A').replace('/', '%2F')
mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
headers = {"Authorization": f"Bearer {bearer_token}"}

# Connect to the deployed MCP server
print("Connecting to MCP server...")
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url, headers))


# Create and test the comprehensive Strands Agent
#with exa_server, websearch_server:
#    mcp_tools = (exa_server.list_tools_sync() + websearch_server.list_tools_sync())
with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"Available MCP tools: {[tool.tool_name for tool in mcp_tools]}")
    agent = Agent(
        model="us.amazon.nova-pro-v1:0",
        system_prompt = """You are a helpful assistant that provides concise responses.
                        If web search, always use websearch tool first.
                        """,
        #system_prompt="""你是一位乐于助人的助手，能提供简洁的回复。
        #                如果要进行网页搜索，请务必先使用websearch工具。""",
        tools=[weather, calculator, agentcore_code_interpreter.code_interpreter] + mcp_tools,
    )

@app.entrypoint
async def strands_agent_bedrock(payload, context):
    """
    Invoke the agent with a payload
    """
    print(f"Payload: {payload}")
    print(f"Context: {context}")
    user_input = payload.get("prompt", "No prompt found")
    stream = agent.stream_async(user_input)
    async for event in stream:
        if "data" in event:
            yield event
    print(agent.messages)

if __name__ == "__main__":
    app.run()

## 幕后发生了什么？

当您使用 `BedrockAgentCoreApp` 时，它会自动：

* 创建一个监听端口 8080 的 HTTP 服务器
* 实现处理代理要求所需的 `/invocations` 端点
* 实现用于健康检查的 `/ping` 端点（对异步代理非常重要）
* 处理适当的内容类型和响应格式
* 根据 AWS 标准管理错误处理

### 本地测试（可选）

在部署到 AgentCore Runtime 之前，您可以在本地测试代理以验证功能。

**启动本地服务器：**
```bash
uv run strands_agent.py
```
或
```bash
python strands_agent.py
```
服务器将在 `http://localhost:8080` 启动

**使用 cURL 测试：**
```bash
curl -X POST http://localhost:8080/invocations \
  -H "Content-Type: application/json" \
  -H "X-Amzn-Bedrock-AgentCore-Runtime-User-Id: 123" \
  -H "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id: 1234567890123456789012345678901234567890" \
  -d '{"prompt": "How is the weather in HK?"}'
```

### 步骤 2：创建依赖文件

定义 Strands Agent 部署所需的 Python 依赖项。

**关键依赖项：**
- **aws-opentelemetry-distro**：AgentCore 可观测性所需
- **strands-agents**：核心 Strands 框架
- **bedrock-agentcore**：运行时集成

**可观测性集成：**
`aws-opentelemetry-distro` 库启用自动检测以进行监控和跟踪。如 [AgentCore 可观测性指南](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability-configure.html) 中所述，生成的 Dockerfile 包含：

```dockerfile
CMD ["opentelemetry-instrument", "python", "main.py"]
```

这种自动检测方法会自动将 OpenTelemetry SDK 添加到 Python 路径中，以实现全面的可观测性。

In [ ]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
bedrock-agentcore
bedrock-agentcore-starter-toolkit
boto3
aws-opentelemetry-distro>=0.10.0

### 可观测性设置（可选）

**CloudWatch APM 配置：**
启用 CloudWatch APM → 事务搜索，以全面观测和监控已部署的 Strands Agent。在本实验中，将 X-Ray 跟踪索引设置为 100%，以生成所有跟踪摘要进行端到端事务分析。
![bedrock-agentcore-observability-setup](images/observability-setup.png)

**优势：**
- **性能监控**：请求跟踪和响应时间指标
- **错误跟踪**：全面的调试功能
- **使用分析**：模式和使用洞察
- **分布式跟踪**：端到端请求流可见性

### 步骤 3：配置 AgentCore Runtime

设置 Bedrock AgentCore Runtime 配置并自动创建资源。

**生成的工件：**
此步骤创建必要的部署文件：
- **Dockerfile**：Strands Agent 的容器配置
- **.dockerignore**：列出 docker build 时排除的文件
- **.bedrock_agentcore.yaml**：运行时部署配置


In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3

region = boto3.session.Session().region_name
agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_getting_started"
)
response

### 步骤 4：部署到 AgentCore Runtime

使用 AWS CodeBuild 启动部署过程进行容器化和部署。

**部署过程：**
- 构建 Strands Agents 的容器化版本
- 创建所需的 AWS 资源（ECR 存储库、IAM 角色）
- 将容器镜像推送到 Amazon ECR
- 作为托管的自动扩展服务部署到 AgentCore Runtime

In [ ]:
launch_result = agentcore_runtime.launch()

### 步骤 5：验证部署状态

监控部署进度并等待运行时准备就绪：

In [ ]:
import time

print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

strands_runtime_id = launch_result.agent_id
strands_runtime_arn = launch_result.agent_arn
strands_ecr_repo_name = launch_result.ecr_uri.split('/')[1]
print(f"Strands AgentCore Runtime ID: {strands_runtime_id}")
print(f"Strands AgentCore Runtime ARN: {strands_runtime_arn}")
print(f"ECR Repo for Strands AgentCore Runtime: {strands_ecr_repo_name}")

## 测试已部署的代理

通过 Bedrock AgentCore Runtime API 调用已部署的 Strands Agent，使用各种提示来验证所有集成工具是否正常工作。

In [ ]:
import boto3
import json
import uuid

#PROMPT = "Search for a coffee maker on amazon.com and extract details of the first one"
#PROMPT = "How is the weather in HK?"
PROMPT = "北京的天气怎么样？"

agent_runtime_arn = launch_result.agent_arn
session_id = str(uuid.uuid4())

agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=boto3.session.Session().region_name
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_runtime_arn,
    qualifier="DEFAULT",
    runtimeUserId="123",
    runtimeSessionId=session_id, #Provide same session identifier across multiple requests to maintain conversation context, and with better traceability
    payload=json.dumps({"prompt": PROMPT})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                print(line)
else:
    events = []
    for event in boto3_response.get("response", []):
        events.append(event)

### 测试对话历史

验证代理使用相同的会话 ID 在多个请求中维护对话上下文：

In [ ]:
PROMPT = "我问你什么了？"

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_runtime_arn,
    qualifier="DEFAULT",
    runtimeUserId="123",
    runtimeSessionId=session_id, #Provide same session identifier across multiple requests to maintain conversation context, and with better traceability
    payload=json.dumps({"prompt": PROMPT})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                print(line)
else:
    events = []
    for event in boto3_response.get("response", []):
        events.append(event)

### 对话历史和会话管理

**Strands Agents 对话管理：**
Strands Agents 包含内置的对话管理，默认使用 `SlidingWindowConversationManager` 策略。这会自动维护会话内的对话上下文。

**参考：** [Strands Agents 对话管理](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/)

**AgentCore Runtime 会话隔离：**
- **会话隔离**：每个用户会话都是隔离和安全的
- **上下文重用**：对话历史在多次调用中保持
- **会话持续时间**：临时会话最长持续 8 小时
- **自动清理**：会话在过期后自动清理

**参考：** [AgentCore Runtime 会话](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-sessions.html)

**长期记忆：**
对于超出会话持续时间的持久记忆，请与 Bedrock AgentCore Memory 集成以进行长期对话存储和检索。

## Amazon CloudWatch 上的 AgentCore 可观测性

### 什么是 Bedrock AgentCore 可观测性？

Amazon Bedrock AgentCore 通过 CloudWatch 和 X-Ray 集成提供内置的可观测性。这使得能够监控代理性能、跟踪请求流程和分析对话模式。

主要功能：
- **会话跟踪**：监控个别对话
- **分布式跟踪**：跨组件跟踪请求
- **性能指标**：延迟、吞吐量和错误率
- **跨度分析**：详细的执行分解

了解更多：[AgentCore 可观测性指南](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)

### 查看主仪表板

访问 CloudWatch 控制台查看您的 AgentCore 可观测性仪表板：
![observability-main-dashboard.png](images/observability-main-dashboard.png)

### 会话管理

在 `strands_claude_getting_started` 中点击 **DEFAULT** 查看会话历史。这显示了我们的测试会话，包含两个跟踪："香港的天气怎么样？" 和 "我问你什么了？"
![observability-session-list.png](images/observability-session-list.png)

### 会话概览

选择一个会话以查看指标、跟踪时间线和性能数据：
![observability-session-overview.png](images/observability-session-overview.png)

### 跟踪分析

点击任何跟踪以检查详细的执行步骤：

**跟踪摘要**：
![observability-trace-summary.png](images/observability-trace-summary.png)

**跨度详情**：
![observability-trace-span-1.png](images/observability-trace-span-1.png)
![observability-trace-span-2.png](images/observability-trace-span-2.png)

## 资源清理（可选）

⚠️ **警告**：这将删除研讨会期间创建的所有资源。只有在您不再需要已部署的代理时才继续。

**要清理的资源：**
- **Strands Agent Runtime**：已部署的多工具代理
- **MCP Server Runtime**：实验 3 中的网络搜索 MCP 服务器
- **ECR 存储库**：两个运行时的容器镜像
- **Cognito 用户池**：身份验证基础设施
- **AgentCore Identity**：实验 2 中的 API 密钥提供程序

**清理过程：**
脚本将尝试按正确顺序删除资源以避免依赖冲突。

In [ ]:
import boto3
import os

region = boto3.session.Session().region_name

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr',region_name=region)
cognito_client = boto3.client('cognito-idp', region_name=region)

try:
    print("Deleting AgentCore Runtime for Strands...")
    agentcore_control_client.delete_agent_runtime(agentRuntimeId=strands_runtime_id)
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository for Strands...")
    ecr_client.delete_repository(repositoryName=strands_ecr_repo_name, force=True)
    print("✓ ECR repository deleted")

    print("Deleting AgentCore Runtime for MCP...")
    agentcore_control_client.delete_agent_runtime(agentRuntimeId=mcp_runtime_id)
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository for MCP...")
    ecr_client.delete_repository(repositoryName=mcp_ecr_repo_name, force=True)
    print("✓ ECR repository deleted")

    print("Deleting Cognito User Pool...")
    cognito_client.delete_user_pool(UserPoolId=cognito_pool_id)
    print("✓ Cognito User Pool deleted")

    print("Deleting AgentCore Identity...")
    agentcore_control_client.delete_api_key_credential_provider(name="exa-apikey-provider")
    print("✓ AgentCore Identity deletion initiated")

    print("Deleting Bedrock AgentCore configuration file for Strands...")
    os.remove(".bedrock_agentcore.yaml") 
    print("✓ .bedrock_agentcore.yaml deleted")

    print("Deleting Bedrock AgentCore configuration file for MCP...")
    os.remove("../04-bedrock-agentcore-runtime-mcp/.bedrock_agentcore.yaml") 
    print("✓ .bedrock_agentcore.yaml deleted")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")